### Query Likelihood model with laplace smoothing. 

- This notebook implements query likelhood model to calculate the probability of a document given a query, using frequencies of query terms withing documents to estimate this likelihood.

## Read the documents and queries.
 - The 1400 documents from the "cran.all.1400.xml" is read and a dictionary containing document id and document text is saved as a dictionary.
 - The queries are also read in similar way and a dictionary with query id and the query is created as  a key-value pair.

In [1]:
# import necessary libraries

import xml.etree.ElementTree as ET
import csv
from tqdm import tqdm
 
# utilities
from collections import defaultdict
import re
import numpy as np
import pandas as pd
from copy import deepcopy
import os
from itertools import chain
import operator
# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize.treebank import (TreebankWordTokenizer, TreebankWordDetokenizer)
from string import punctuation
from nltk import PorterStemmer
from collections import Counter

import math
from functools import lru_cache
import numpy as np
import math
import operator
from tqdm import tqdm
import itertools
from collections import Counter
from dataclasses import dataclass
from math import log
import pickle
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from num2words import num2words
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)


True

In [2]:
''' 
The class ReadDocs parses the input .xml file and creates a dictionary with docid and text as key-value pairs.

The class ReadQueries parses the input .xml file and creates a dictionary with queryid and text as key-value pairs.
'''
class ReadDocs:
    def __init__(self, input_file):
        self.input_file = input_file
        self.split_tag = "</doc>"
        self.documents = []
        
    def load_split_docs(self):
        ''' 
        Read the file and split the documents based on the split_tag.

        return: list of documents
        '''
        with open(self.input_file, "r") as f:
            data = f.read()
        documents_raw = data.split(self.split_tag)
        self.documents = [doc.strip() + self.split_tag for doc in documents_raw if doc.strip()]
    
    def read_document(self):
        ''' 
        Read the documents using ElementTree and create a dictionary with docid and text as key-value pairs.
        
        return: dictionary with docid and text as key-value pairs
        '''
        self.doc_dict = {}
        for _, doc in enumerate(self.documents):
            try:
                root = ET.fromstring(f"<root>{doc}</root>")
                docno_element = root.find('.//docno')
                text = root.find('.//text')
                docno = docno_element.text.strip() if docno_element is not None and docno_element.text else "**NODATA**"
                text = text.text.strip() if text is not None and text.text else "**NODATA**"
                if text == "**NODATA**":
                    continue
                else:
                    text = text.replace("\n", " ").replace(" .", "")
                    self.doc_dict[int(docno)] = str(text)
            except ET.ParseError as e:
                continue
        return self.doc_dict

class ReadQueries:
    def __init__(self, input_file):
        self.input_file = input_file
        self.queries = []
    
    def load_split_query(self):
        '''Read the file and split the queries based on the split_tag.
        
        return: Dictionary of queryid and query text as key-value pair
        '''        
        with open(self.input_file, "r") as f:
            data = f.read()
        self.queries = data.split("</top>")
        self.queries = [query.strip() + "</top>" for query in self.queries if query.strip()]
        
    def index_queries(self):
        query_dict = {}
        for i, query in enumerate(self.queries):
            try:
                root = ET.fromstring(f"<root>{query}</root>")
                num = root.find('.//num').text.strip()
                title = root.find('.//title').text.strip()
                title = title.replace("\n", " ").replace(" .", "")
                query_dict[int(num)] = str(title)
            except ET.ParseError as e:
                continue
        return query_dict
    
def mapping(query, doc):        
    mappings = {}
    for key, value in query.items():
        for k, v in doc.items():
            doc_keys = list(doc.keys())
            doc_keys = [int(i) for i in doc_keys]
            mappings[int(key)] = doc_keys 
    return mappings

# Text Preprocessing: 
- The text is preprocessed to tokenise a document.
- Temove stop words which are mostly commonly used in English text.
- Convert numbers to it word representation. 
- Normalise text by lowercasing and string with only alphabets.
- Stemming is performed to reduce the word to their root form.



In [3]:
class PreprocessText:
    def __init__(self, text):
        self.text = text
        
    def _tokenize(self):
        ''' 
        - tokenise the text using word_tokenize function.
        
        '''
        tokenizer = TreebankWordTokenizer()
        self.tokens = tokenizer.tokenize(self.text)  
        return self.tokens
    
    def _remove_stopwords(self):
        stop_words = set(stopwords.words('english'))
        return [word for word in self.tokens if word.lower() not in stop_words]
    
    def _convert_num_text(self):
        text_num = [num2words(word) if word.isnumeric() and word.isascii() else word for word in self.tokens]
        return text_num
    
    def _normalise_text(self):
        normalised_text = [word.lower() for word in self.tokens if word.isalpha()]
        return normalised_text  
    
    def _stem_text(self):
        stemmer = PorterStemmer()
        return [stemmer.stem(word) for word in self.tokens]
    
    def clean_text(self):
        self._tokenize()
        self.tokens = self._convert_num_text()
        self.tokens = self._normalise_text()
        self.tokens = self._remove_stopwords()
        self.tokens = self._stem_text()
        return self.tokens  

# Inverted Index
- Inverted index of the document is created as a dictionary where each preprocessed term key and the value is a key-value pair of document id and posting information of term frequency and IDF along with document frequency for the term.

In [4]:
@dataclass   
class Posting:
    freq: int = 1
    tfidf: float = 0.0
    
class InvertedIndexDocs:
    def __init__(self, documents):
        self._documents = self._clean_data(documents)
        self._indexdocs = dict()
        
    def _clean_data(self, document_list):
        clean_docs = {doc_id: PreprocessText(doc_text).clean_text() for doc_id, doc_text in document_list.items()}
        return clean_docs
    
    def create_index(self):
        self._index_docs()
        self._tfidf_docs()
        
    def _index_docs(self):
        for docid, docs in self._documents.items():
            for token in docs:
                if token not in self._indexdocs:
                    self._indexdocs[token] = InvertList()
                self._indexdocs[token].add_posting_doc(docid)
    
    def _tfidf_docs(self):
        for token, inv_index in self._indexdocs.items():
            for _, posting in inv_index.postings.items():
                term_freq = posting.freq
                idf = log(self.document_length / self._indexdocs[token].document_frequency)
                posting.tfidf = term_freq * idf
                
    @property
    def get_index(self):
        indexed_doc = {k:v for k,v in sorted(self._indexdocs.items())}
        return indexed_doc
    
    @property
    def documents_collection(self):
        return self._documents

    @property
    def words(self):
        return list(itertools.chain.from_iterable(self._documents.values()))

    @property
    def vocab(self):
        return sorted(self._indexdocs)

    @property
    def document_length(self):
        return len(self._documents)

    @property
    def word_count(self):
        return len(self.words)

    @property
    def vocab_count(self):
        return len(self._indexdocs)

    @property
    def avg_doc_length(self):
        return self.word_count / self.document_length

    @property
    def counter(self):
        return Counter(self.words)
    
class InvertList:
    def __init__(self):
        self._postings = dict()

    def add_posting_doc(self, docid):
        if self.contains_posting_data(docid):
            return self.update_posting(docid)
        posting = Posting()
        self._postings[docid] = posting

    def update_posting(self, docid):
        if not self.contains_posting_data(docid):
            return self.add_posting_doc(docid)
        posting = self._postings[docid]
        posting.freq += 1

    def get_posting(self, docid):
        return self._postings[docid]

    def contains_posting_data(self, docid):
        return docid in self._postings

    @property
    def postings(self):
        return self._postings

    @property
    def document_frequency(self):
        return len(self._postings)

# BM25 Model

In [5]:
class RunModel:
    def __init__(self, index, mapping):
        self._document_collection = index.documents_collection
        self._index = index.get_index
        self._mapping = mapping
        
    def rank(self, qid, query):
        query_words = self._clean_query_text(query)
        passages = self._relevant_docs(qid, query_words)

        scores = {}
        for docid in passages:
            scores[docid] = self._score_docs(docid, query_words)

        ranking = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        ranks = {docid: score for docid, score in ranking}
        return ranks

    def _clean_query_text(self, query):
        clean_query_text = [word for word in PreprocessText(query).clean_text()]
        return clean_query_text

    def _relevant_docs(self, qid, query_words):
        relevant_docs_set = set()
        for word in query_words:
            if word in self._index:
                for docid in self._index[word].postings:
                    if docid in self._mapping[qid]:
                        relevant_docs_set.add(docid)
        return list(relevant_docs_set)

class BM25(RunModel):
    def __init__(self, index, mapping):
        super().__init__(index, mapping)
        self._doc_length = index.document_length
        self._avg_length = index.avg_doc_length

    def _score_docs(self, docid: int, query_words) -> float:
        BM25_score = 0
        for word in query_words:
            if word not in self._index or not self._index[word].contains_posting_data(docid):
                continue
            BM25_score += self._score(docid, word)
        return BM25_score

    def _score(self, docid: int, word: str) -> float:
        # Constants
        k1 = 1.2
        b = 0.75

        # Parameters
        inv_index = self._index[word]
        doc_length = self._doc_length
        doc_freq = inv_index.document_frequency
        tf = inv_index.get_posting(docid).freq
        dl = float(len(self._document_collection[docid])) 
        avg_doc_length = float(self._avg_length)

        # Formulas
        K = k1 * ((1 - b) + b * (dl / avg_doc_length))
        first_exp = log(((doc_length - doc_freq + 0.5) / (doc_freq + 0.5)) + 1)
        second_exp = ((tf * (k1 + 1)) / (tf + K))
        return first_exp * second_exp



# VSM

In [6]:
class VSM(RunModel):
    def __init__(self, index, mapping):
        super().__init__(index, mapping)
        self._vocab = index.vocab
        self._collection_length = index.document_length
        self._vocab_count = index.vocab_count

    def _score_docs(self, docid: int, query_words) -> float:
        vocab = list(set(self._document_collection[docid]))
        vocab_count = len(vocab)

        doc_vector = np.zeros(vocab_count)
        for i, word in enumerate(vocab):
            doc_vector[i] = self._index[word].get_posting(docid).tfidf

        query_vector = np.zeros(vocab_count)
        counter = Counter(query_words)
        max_freq = counter.most_common(1)[0][1]
        for word in query_words:
            if word not in self._index:
                continue
            tf = (0.5 + (0.5 * (counter[word] / max_freq)))
            idf = log(self._collection_length / self._index[word].document_frequency)
            tfidf = tf * idf
            if word in vocab:
                wordidx = vocab.index(word)
                query_vector[wordidx] = tfidf
            else:
                query_vector = np.append(query_vector, tfidf)
                doc_vector = np.append(doc_vector, 0)
                
        cosine_similarity = np.dot(doc_vector, query_vector) / (np.linalg.norm(doc_vector) * np.linalg.norm(query_vector))
        return cosine_similarity

# Query likelihood model with Laplace Smoothing

In [7]:
class LanguageModel(RunModel):
    def __init__(self, index, mapping):
        super().__init__(index, mapping)
        self._all_words = index.words
        self._word_count = index.word_count
        self._vocab_count = index.vocab_count
        self._counter = index.counter
        
    def _score_docs(self, docid: int, query_words) -> float:
        prob = []
        for token in query_words:
            prob.append(self._calc_probability(docid, token))
        try:
            return log(np.prod(prob))
        except ValueError:
            return 0.0
    
    def _calc_probability(self, docid: int, token: str) -> float:
        term_freq = self._index[token].get_posting(docid).freq if token in self._document_collection[docid] else 0
        document_length = len(self._document_collection[docid])
        vocab = self._vocab_count
        probability = (term_freq + 1) / (document_length + vocab)
        return probability


In [8]:
input_docs = r'./data/cran.all.1400.xml'

read_documents = ReadDocs(input_docs)
read_documents.load_split_docs()
cran_docs = read_documents.read_document()

query_path = r'./data/cran.qry.xml'
read_query = ReadQueries(query_path)
read_query.load_split_query()
cran_queries = read_query.index_queries()

def generate_index(doc_collection):
    index = InvertedIndexDocs(doc_collection)
    index.create_index()
    return index

inv_index = generate_index(cran_docs)

for k, v in inv_index.get_index.items():
    print(f'the term is {k}, posting list is {v.postings} and document frequency is {v.document_frequency}')

the term is ab, posting list is {744: Posting(freq=1, tfidf=7.242797922793756)} and document frequency is 1
the term is abbrevi, posting list is {122: Posting(freq=1, tfidf=7.242797922793756)} and document frequency is 1
the term is abil, posting list is {51: Posting(freq=1, tfidf=6.144185634125646), 77: Posting(freq=1, tfidf=6.144185634125646), 738: Posting(freq=1, tfidf=6.144185634125646)} and document frequency is 3
the term is abl, posting list is {99: Posting(freq=1, tfidf=4.94021282979971), 132: Posting(freq=1, tfidf=4.94021282979971), 536: Posting(freq=1, tfidf=4.94021282979971), 581: Posting(freq=1, tfidf=4.94021282979971), 695: Posting(freq=1, tfidf=4.94021282979971), 763: Posting(freq=1, tfidf=4.94021282979971), 908: Posting(freq=1, tfidf=4.94021282979971), 914: Posting(freq=1, tfidf=4.94021282979971), 986: Posting(freq=1, tfidf=4.94021282979971), 1114: Posting(freq=1, tfidf=4.94021282979971)} and document frequency is 10
the term is ablat, posting list is {82: Posting(freq=2

In [ ]:

#------------------------------------------------------- Execute this if mapping is not found.------------------------------
'''
map_doc_queries = mapping(cran_queries, cran_docs)

# save map_doc_queries as pickle
import pickle
with open('map_doc_queries.pkl', 'wb') as f:
    pickle.dump(map_doc_queries, f)'''

# read map_doc_queries.pkl
with open('map_doc_queries.pkl', 'rb') as f:
    mapped_data = pickle.load(f)
    
################################## BM25 ##################################
model = BM25(inv_index, mapped_data)
ranking = {qid: model.rank(qid, query) for qid, query in cran_queries.items()}

filename = r"./results/bm25_ranking.txt"  
with open(filename, "w") as file:
    for qid, rankings in ranking.items():
        for rank, (docid, score) in enumerate(rankings.items(), start=1):
            file.write(f"{qid} Q0 {docid} {rank} {score:.4f} RUN_ID\n")
            
################################## VSM ##################################
# run VSM model
model = VSM(inv_index, mapped_data)

# loop through each query and perform ranking
ranking = {qid: model.rank(qid, query) for qid, query in cran_queries.items()}

# save the ranking in a text file.
filename = r"./results/vsm_ranking.txt"
with open(filename, "w") as file:
    for qid, rankings in ranking.items():
        for rank, (docid, score) in enumerate(rankings.items(), start=1):
            # select only top 100 ranks
            file.write(f"{qid} Q0 {docid} {rank} {score:.4f} RUN_ID\n")
            

################################## Query Likelihood with Laplace smoothing language Model ##################################
# run Language model

model = LanguageModel(inv_index, mapped_data)

# perform ranking
ranking = {qid: model.rank(qid, query) for qid, query in cran_queries.items()}

# save the result to a file as query_id iter <space> document_id <space> rank <space> similarity <space> run_id
filename = r"./results/language_model.txt"
with open(filename, "w") as file:
    for qid, rankings in ranking.items():
        for rank, (docid, score) in enumerate(rankings.items(), start=1):
            file.write(f"{qid} Q0 {docid} {rank} {score:.4f} RUN_ID\n")